In [2]:
import os
import tweepy 
import pandas as pd


In [3]:
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAAafZwEAAAAA9T7RuHKiPM8nH%2BSmt7JkyvAwG%2Bc%3DuU2nNckjJw7966aZPXo0sr4ZBSvCLVR43OjZyAPrlKCjOeDjBe'

In [4]:
auth = tweepy.OAuth2BearerHandler(bearer_token)
api = tweepy.API(auth,wait_on_rate_limit=True)


In [5]:
client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True )

In [73]:
search_q='url:"https://www.sciencedirect.com" OR url:"https://ieeexplore.ieee.org" OR url:"https://arxiv.org/" has:links -congratulations -"more here" -colleagues -please -our -entity:"COVID19"'

In [74]:
search_q

'url:"https://www.sciencedirect.com" OR url:"https://ieeexplore.ieee.org" OR url:"https://arxiv.org/" has:links -congratulations -"more here" -colleagues -please -our -entity:"COVID19"'

In [75]:
user_ids=[]
tweets=[]
tweet_urls=[]
tweet_ids=[]
is_retweet=[]


In [76]:
for tweet in tweepy.Paginator(client.search_recent_tweets,search_q,expansions=["author_id"],tweet_fields=["entities","text","referenced_tweets"]).flatten(limit=5):
        isRetweet=False
        tweets.append(tweet.text)
        ent=tweet['entities']
        try:
            retweets_info=tweet.data["referenced_tweets"]
            ret_ids=[]
            isRetweet=True
            for r in retweets_info:
                ret_ids.append(r['id']) # dodaj retweet id
        except:
            pass #not retweet
        
        tweet_ids.append(tweet.id)
        user_ids.append(tweet.author_id)
        user_urls=[] #v tvitu more bit više neg jeden link
        try:
            all_tw_urls=ent['urls']
            for u in all_tw_urls:
                try:
                    user_urls.append(u['unwound_url']) # manje šanse da je opet skraćeni link tipa ".ly"
                except:
                    user_urls.append(u['expanded_url'])
            
        except KeyError: # samo ako je retweet, u tom slučaju je neki problem sažimanja koji sam rješila dodatnim korakom ispod
            pass
        if isRetweet and len(user_urls)==0:
            tweet_urls.append(ret_ids)
            is_retweet.append(isRetweet)

        else:  
            tweet_urls.append(user_urls)
            is_retweet.append(False) #ako je tweet ili retweet ali koji ima linkani članak



In [81]:
retweets_info,tweet.id,tweet.text

([{'type': 'retweeted', 'id': '1504631666131697664'}],
 1504937265247789059,
 'https://t.co/vg31Xs1CGM')

In [77]:
df1=pd.DataFrame(
    {'userID': user_ids,
     'tweet': tweets,
     'tweetID': tweet_ids,
     'source':tweet_urls,
     'isRetweet':is_retweet
    })
df1.head()

,userID,tweet,tweetID,source,isRetweet
0,51389580,"When I became an ophthalmologist, I was told t...",1504937331039805441,[(https://www.sciencedirect.com/science/articl...,False
1,2451725746,RT @ak92501: Latent Image Animator: Learning t...,1504937283073548303,"[(https://arxiv.org/abs/2203.09043, un)]",False
2,892322070,RT @sandralopezleon: 🦠 Este artículo nombra al...,1504937268116692998,[1504820127556116480],True
3,1089590856640851970,RT @pzhong2016: Anatomy of the LV summit and i...,1504937267470770177,[(https://www.sciencedirect.com/science/articl...,False
4,1488691684703707137,https://t.co/vg31Xs1CGM,1504937265247789059,"[(https://www.sciencedirect.com/, un)]",False


In [94]:
data.head()

,userID,tweet,tweetID,source,isRetweet
0,718327719453401088,RT @EARL_COVID19_tw: ちなみに、北里が行ったコロナに対するイベルメクチン...,1504766158892806144,['1504683684309307394'],True
1,797260867310092288,RT @arxiv_cs_LG: Ranking of Communities in Mul...,1504766131378208770,['1504672717999464448'],True
2,1151202637582348294,RT @VCCLibrary: New first-line #immunotherapy ...,1504766113904795648,['1504766092081774594'],True
3,1023684972681592838,RT @phd_sean: @romulo_ef @David_johnson11 @sta...,1504766092228636683,['1504532651239215104'],True
4,1288296943,New first-line #immunotherapy -based combinati...,1504766092081774594,[('https://www.sciencedirect.com/science/artic...,False


In [14]:
len(tweet_urls)==len(tweet_ids)

True

In [15]:
df1.to_csv('datasets/twitter/twitter_18_3.csv',index=False)

In [16]:
len(tweet_ids)

54559

userID |	tweet          |	tweetID |	source   | isRetweet

-> 15161684 | "blablabla"  | 16846846  | [url1, url2] |  False

ILI

-> 15616516 | "blablabla" | 468468464  | 1856816816 |  True

iz čega će slijediti,

-> 15616516 | "blablabla" | 468468464   | [url1] |  True

gdje je *url1* url iz originalnog tweeta

\+ 18681618 | "blablabla url1" | 468468464 | [url1]  | False 


##### Dakle, pokupili smo 
* tweet sa linkom
* retweet sa linkom koji je linkan u originalnom tweetu
* originalni tweet koji je retweetan
kao i pripadne korisnike koji su objavili tweet ili retweet

### Neki retweetovi ne sadrže link, tako da u narednom koraku tražimo originalni tweet koji bi trebao imati neki link na arxiv ili ostale...

In [8]:
data = pd.read_csv('datasets/twitter/twitter_18_3.csv')

In [92]:
#tweet_row: ret_from
#5241:456654654654
retw_ids=[]

for i,row in data.iterrows():
    if row.isRetweet:
            retw_ids.append(str(row.source[2:-2]))
            break


In [93]:
retw_ids

['1504683684309307394']

In [100]:
start=0
end=100
step=100
tw_urls=dict()

In [101]:
while(end < len(retw_ids)):
    tweets_info=client.get_tweets(retw_ids[start:end],expansions=["author_id"],tweet_fields=["entities","text","referenced_tweets"]) # *, expansions=None, media_fields=None, place_fields=None, poll_fields=None, tweet_fields=None, user_fields=None, user_auth=False)
    for tweet in tweets_info.data:
        try:
            urls=tweet["entities"]['urls']
            all_urls=[]
            for u in urls:
                try:
                    all_urls.append(u["unwound_url"]) #manje sanse da se radi o bit.ly linku
                except:
                    all_urls.append(u["expanded_url"])
            tw_urls[str(tweet.id)]=all_urls
        except:
            print(tweet.text)
    start+=step
    end+=step

In [ ]:
n=0
for i,row in data.iterrows():
    if row.isRetweet:
        try:
            data.loc[i, 'source'] = tw_urls[row.source[2:-2]]
        except:
            print(row)


In [104]:
data.head()

,userID,tweet,tweetID,source,isRetweet
0,718327719453401088,RT @EARL_COVID19_tw: ちなみに、北里が行ったコロナに対するイベルメクチン...,1504766158892806144,https://doi.org/10.1016/j.jiac.2021.08.021,True
1,797260867310092288,RT @arxiv_cs_LG: Ranking of Communities in Mul...,1504766131378208770,http://arxiv.org/abs/2203.09281,True
2,1151202637582348294,RT @VCCLibrary: New first-line #immunotherapy ...,1504766113904795648,http://ow.ly/qk2P50Imsru,True
3,1023684972681592838,RT @phd_sean: @romulo_ef @David_johnson11 @sta...,1504766092228636683,https://www.sciencedirect.com/science/article/...,True
4,1288296943,New first-line #immunotherapy -based combinati...,1504766092081774594,[('https://www.sciencedirect.com/science/artic...,False


In [108]:
data.to_csv('datasets/twitter/twitter_18_3_all.csv',index=False)

In [109]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54559 entries, 0 to 54558
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userID     54559 non-null  int64 
 1   tweet      54559 non-null  object
 2   tweetID    54559 non-null  int64 
 3   source     54559 non-null  object
 4   isRetweet  54559 non-null  bool  
dtypes: bool(1), int64(2), object(2)
memory usage: 1.7+ MB
